This NB follows through the reflection algorithm in the reflection layer.

In [ ]:
import numpy as np
import tensorflow as tf

# Shapes

Go through the algorithm with dummy data to check broadcasting, shapes, etc.. Compare to <code>src/reflection_layer.py</code>.

## Prepare dummy data

In [ ]:
# set up array (aka tensor) shapes for experiments
data_shape = (50, 8, 8, 3)  # shape of the full data set
sample_shape = data_shape[1:]  # shape of a single sample

In [ ]:
# define reflector (ie kernel)
reflector = np.random.rand(*sample_shape)
print(reflector.shape)

In [ ]:
# define data
x = np.random.rand(*data_shape)
print(x.shape)

## Steps of the algorithm

In [ ]:
# elementwise product of reflector and data (careful: broadcasting!)
ewp = tf.multiply(x,reflector) 
ewp.shape

In [ ]:
# reduce sum of ewp gives vector of scalar products: samples dot reflector
scalars = tf.reduce_sum(ewp, axis=tuple(range(1, len(x.shape))), keepdims=True)
scalars.shape

In [ ]:
# expand dimensions of reflector kernel so that it can be multiplied with scalars
expanded_reflector = tf.expand_dims(reflector, 0)
expanded_reflector.shape

In [ ]:
# (broadcasting!)
temp = np.multiply(expanded_reflector, scalars)
temp.shape

In [ ]:
# final output: reflection of data x across the orthogonal complement of reflector
output = (2/ tf.reduce_sum(tf.multiply(reflector, reflector))) * temp - x
output.shape

# Explicit example

Check for an explicit example that the algorithm does the right thing.

## Function

In [ ]:
def reflect(x, reflector):
    # elementwise product of reflector and data (careful: broadcasting!)
    ewp = tf.multiply(x,reflector) 
    # reduce sum of ewp gives vector of scalar products: samples dot reflector
    scalars = tf.reduce_sum(ewp, axis=tuple(range(1, len(x.shape))), keepdims=True)
    # expand dimensions of reflector kernel so that it can be multiplied with scalars
    expanded_reflector = tf.expand_dims(reflector, 0)
    # (broadcasting!)
    temp = np.multiply(expanded_reflector, scalars)
    # final output: reflection of data x across the orthogonal complement of reflector
    return (2 / tf.reduce_sum(tf.multiply(reflector, reflector))) * temp - x

## Data

In [ ]:
reflector=tf.convert_to_tensor(np.array([1,0,0]), dtype=tf.float32)
x = tf.convert_to_tensor(np.array([[1,0,0],[0,1,0],[0,0,1],[1,1,1]]), dtype=tf.float32)

## Result

In [ ]:
with tf.Session() as sess:
    print('As the reflection of')
    print(x.eval())
    print('across the orthogonal complement of')
    print(reflector.eval())
    print('(followed by an overall multiplication by -1) the algorithm obtains')
    print(reflect(x, reflector).eval())
    print("\nYep, that's what it should be.")